notes to myself:
* pf
    - 2step_free_F17
    - 2step_controlled_F16
    - 1step_free_F14
    - 1step_controlled_F17
* NCP
    - 2step_free_F16
    - 1step_controlled_F16
    - 1step_controlled_F14

In [1]:
%matplotlib qt5

import numpy as np
import matplotlib.pyplot as plt
from matplotlib import style
import pickle

In [2]:
with open('./log/results/meta_results_pf_2step_F17.pickle','rb') as f:
    results_2step_F17 = pickle.load(f)
with open('./log/results/meta_results_pf_2step_F16.pickle','rb') as f:
    results_2step_F16 = pickle.load(f)
with open('./log/results/meta_results_pf_1step_F17.pickle','rb') as f:
    results_1step_F17 = pickle.load(f)
with open('./log/results/meta_results_pf_1step_F14.pickle','rb') as f:
    results_1step_F14 = pickle.load(f)

In [3]:
scenario = {'2step':results_2step_F17}
scenario = {'2step':results_2step_F17,'1step_controlled':results_1step_F17}
# scenario = {'2step':results_2step_F17,'1step_free':results_1step_F14}
scenario = {'2step':results_2step_F17,'1step_controlled':results_1step_F17,'1step_free':results_1step_F14}

In [4]:
tray_flag = [i for i in range(1,21)]
feed_flag = [8,9,10,11,12,13,14,15,16,17,18,19,20]
reactive_flag = [8,9,10,11,12,14,15,16,17,18]
product_flag = ['condenser','reactive[4]','reactive[7]','reactive[10]','reboiler']

# Create an easy-to-carry dictionary

In [5]:
obj_data = {key: [] for key in scenario}
T_data = {key: {j:[] for j in reactive_flag} for key in scenario}
P_data = {key: {j:[] for j in product_flag} for key in scenario}
R_data = {key: [] for key in scenario}
F_data = {key: {j:[] for j in tray_flag} for key in scenario}

for key in scenario:
    for it in scenario[key]['reactive[1].x_[in,H2]'].keys():
        for j in reactive_flag:
            T_data[key][j].append(scenario[key]['reactive[{}].T'.format(j)][it])
        for j in product_flag:
            P_data[key][j].append(scenario[key]['{}.L[P]'.format(j)][it])
        R_data[key].append(scenario[key]['condenser.L[out]'][it]/\
                          scenario[key]['condenser.L[P]'][it])
        
        # fot 2-step approach, first half F is fixed, so value couldn't be found in the result object
        for j in tray_flag:
            try:
                F_data[key][j].append(scenario[key]['reactive[{}].F'.format(j)][it])
            except KeyError:
                if j in reactive_flag:
                    F_data[key][j].append(1)
                else:
                    F_data[key][j].append(0)

# Plot style sheet

In [6]:
style.use('seaborn-paper')

In [7]:
fig, axs = plt.subplots(5,1,figsize=(10,12),sharex=True)

for key in scenario:
    axs[0].plot(P_data[key]['reactive[7]'],label=key)
    axs[1].plot(R_data[key],label=key)
    axs[2].plot(P_data[key]['condenser'],label=key)
    axs[3].plot(P_data[key]['reactive[4]'],label=key)
    axs[4].plot(P_data[key]['reactive[10]'],label=key)

for ax in axs:
    ax.grid()

axs[0].set_title('Obj: Gasoline')
axs[0].legend()

axs[1].set_title('Reflux Ratio')
axs[1].legend()

axs[2].set_title('Naphtha')
axs[2].legend()

axs[3].set_title('Heavy Naphtha')
axs[3].legend()

axs[4].set_title('Diesel')
axs[4].legend()

plt.tight_layout()
plt.show()

In [8]:
fig2, axs = plt.subplots(len(reactive_flag),2,figsize=(10,12),sharex=True)

for j, stage in enumerate(reactive_flag):
    for key in scenario:
        axs[j][0].plot(T_data[key][stage],label=key)
        axs[j][1].plot(np.absolute((np.array(T_data[key][stage]) - T_data[key][stage][-1])/T_data[key][stage][-1]),label=key)
        axs[j][0].set_ylim(200+273.15,300+273.15)
        axs[j][1].set_ylim(0,0.1)

for a in axs:
    for x in a:
        x.grid()        

axs[0][0].set_title('Temperature')
axs[0][1].set_title('Temperature %')
        
axs[0][1].legend()
fig2.suptitle('Temperature')

plt.tight_layout()
plt.show()

In [9]:
fig3, axs = plt.subplots(len(feed_flag),2,figsize=(10,12),sharex=True)

for j, stage in enumerate(feed_flag):
    for key in scenario:
        axs[j][0].plot(F_data[key][stage],label=key)
        axs[j][1].plot(np.absolute((np.array(F_data[key][stage]) - F_data[key][stage][-1])/F_data[key][stage][-1]),label=key)
        axs[j][0].set_ylim(0,6)
        # axs[j][1].set_ylim(0,1)

for a in axs:
    for x in a:
        x.grid()        

axs[0][0].set_title('Feed')
axs[0][1].set_title('Feed %')
        
axs[0][1].legend()
fig3.suptitle('Feed')

plt.tight_layout()
plt.show()